In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from preprocessing import preprocess_text, results_report

from sklearn.model_selection import train_test_split


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression


In [92]:
df = pd.read_csv("Spam Email raw text for NLP.csv")
df.drop('FILE_NAME', axis=1, inplace=True)
df['CATEGORY'] = df['CATEGORY'].replace({1: 'Spam', 0: 'Non Spam'})
class_labels = ["Non Spam", "Spam"]
print(f"Shape: {df.shape}")
df.head(10)

Shape: (5796, 2)


,CATEGORY,MESSAGE
0,Spam,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
1,Spam,ATTENTION: This is a MUST for ALL Computer Use...
2,Spam,This is a multi-part message in MIME format.\n...
3,Spam,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
4,Spam,This is the bottom line. If you can GIVE AWAY...
5,Spam,------=_NextPart_000_00B8_51E06B6A.C8586B31\n\...
6,Spam,"<STYLE type=""text/css"">\n\n<!--\n\nP{\n\n fon..."
7,Spam,<HR>\n\n<html>\n\n<head>\n\n <title>Secured I...
8,Spam,"<table width=""600"" border=""20"" align=""center"" ..."
9,Spam,"<html>\n\n\n\n<head>\n\n<meta http-equiv=""Cont..."


In [93]:
# processed = df['MESSAGE'].head(10).apply(preprocess_text)# apply the preprocess function to the first 200 rows of the dataframe 
# processed

In [94]:
label_encoder = LabelEncoder()# instantiate a label encoder
df['CATEGORY_ENC'] = label_encoder.fit_transform(df['CATEGORY'])# fit and transform the encoder on labels

In [95]:
text_clf = Pipeline([("CountVectorizer", TfidfVectorizer(preprocessor=preprocess_text)), 
                     ("MultinomialNB", MultinomialNB())])

In [96]:
# Split the data into training and testing sets
X = df['MESSAGE']
y = df['CATEGORY_ENC']

test_size=0.2 
random_state=42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

In [97]:
# fit the model
text_clf.fit(X_train, y_train)
# predict and generate classification report
y_pred = text_clf.predict(X_test) # Use the pipeline to run predictions on the test data

# call the report function to visualize the classification results, use the above class_labels
results_report(y_test, y_pred, class_labels)

              precision    recall  f1-score   support

    Non Spam       0.95      0.99      0.97       762
        Spam       0.99      0.90      0.94       398

    accuracy                           0.96      1160
   macro avg       0.97      0.95      0.96      1160
weighted avg       0.96      0.96      0.96      1160

